# BigQuery Embedding and Vector Search

**Text embeddings are a key enabler and building block for applications such as semantic search, recommendation, text clustering, sentiment analysis, and named entity extractions.**

**Embeddings are high-dimensional numerical vectors that represent any entity or object, e.g. text, audio, image etc. With the rise of ML-based models, encoding semantics into embedding vectors has become a common practice for managing and reasoning about content. One common operation in clustering, classification, and recommendation is to measure the distance between vectors to find items that are most semantically similar. This operation is often referred to as vector search.**

**BigQuery vector search lets you use GoogleSQL to do semantic search, using vector indexes for fast but approximate results, or using brute force for exact results.**

*`Note the the following codes are run in a BigQuery Notebook or Colab Enterprise within your GCP project. If you would run the notebook from a different environment, please do the necessary adjustment.`*

### Setting up the connection

Provide the project ID

In [10]:
project_ID = "lohk-da-coaching"  # @param {type:"string"}


Create a remote connection (bash command)

In [ ]:
!bq mk --connection --location=US --project_id={project_ID}  --connection_type=CLOUD_RESOURCE vertex_embeddings2

Retrieve the service account name from the created connection. You can find it by opening up the created connection in the BigQuery console, and look for attribute *Service account id*

In [8]:
connection_service_account = "bqcx-458398081798-ziog@gcp-sa-bigquery-condel.iam.gserviceaccount.com"  # @param {type:"string"}


Grant the role of 'Vertex AI User' to the created connection service account

In [ ]:
!gcloud projects add-iam-policy-binding {project_ID} \
  --member='serviceAccount:{connection_service_account}' \
  --role='roles/aiplatform.user'

### Create an embedding model and generate embeddings
Create a remote embedding model.
ENDPOINT option is preferred over using: ENDPOINTC OPTIONS(remote_service_type = CLOUD_AI_TEXT_EMBEDDING_MODEL_V1');

In [42]:
%%bigquery
CREATE OR REPLACE  MODEL test_embeddings.llm_embedding_model
  REMOTE WITH CONNECTION `us.vertex_embeddings2`
  OPTIONS(ENDPOINT='textembedding-gecko')

Query is running:   0%|          |

""


Generate embeddings for movie reviews (limit to 8000 records) from the public table **bigquery-public-data.imdb.reviews**.

Store the result into a new table called **embedded_reviews**.

In [44]:
%%bigquery
CREATE OR REPLACE TABLE test_embeddings.embedded_reviews
AS SELECT content as review, text_embedding
FROM
  ML.GENERATE_TEXT_EMBEDDING(
    MODEL `test_embeddings.llm_embedding_model`,
    (SELECT review as content
      FROM bigquery-public-data.imdb.reviews limit 8000
    ),
    STRUCT(TRUE AS flatten_json_output)
  );

Query is running:   0%|          |

""


Let's preview the first 10 rows that now contain embedding information. There should be 768 dimension of embeddings per 1 row.

In [45]:

%%bigquery
select * from test_embeddings.embedded_reviews limit 10;


Query is running:   0%|          |

Downloading:   0%|          |

,review,text_embedding
0,"Shallow, shallow script ...stilted acting ...t...","[0.05698924884200096, -0.046766530722379684, -..."
1,This is probably the only movie I have ever no...,"[0.0300773773342371, -0.0564456544816494, -0.0..."
2,"""Return of the Seven"" has a few good action sc...","[0.019662456586956978, -0.018482323735952377, ..."
3,I had to compare two versions of Hamlet for my...,"[0.019407277926802635, -0.020676614716649055, ..."
4,I really don't see how anyone could enjoy this...,"[0.036810245364904404, -0.03499123826622963, -..."
5,"Two days after seeing this thing, I'm still in...","[0.004573755897581577, -0.06571651995182037, -..."
6,for a lot of time I was looking forward to see...,"[0.02161286026239395, -0.0008249685633927584, ..."
7,A woman who deals in art starts to have a pass...,"[0.02168247662484646, -0.017583034932613373, -..."
8,About three minutes into this thing I started ...,"[0.01700712740421295, -0.03861521556973457, -0..."
9,"I went to see this a few days ago, and it's ha...","[0.025232087820768356, -0.01825287751853466, -..."


### Search by computing the distance between two vectors.

Now that we have the embeddings, we can search the nearest neighbour using consine distance by using [ML.DISTANCE function](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-distance). By computing the cosine distance between the query embedding and the embeddings of the search candidate in the predicted cluster, you can get a set of the most similar items to the query item.

This is not the most efficient mechanism: if you compare a vector with 768 elements to 1M vectors, the number of calculations will be proportional to 768 x 1M = 768M. This is the computation required to look through all the entities for a single search, and the reason why the ML.DISTANCE would take longer time to complete.

Let's select all reviews that are ranked closer to the review 'This movie was bad'

In [46]:
%%bigquery
WITH bad_review_embedding AS (
  SELECT text_embedding
  FROM
    ML.GENERATE_TEXT_EMBEDDING(
      MODEL `test_embeddings.llm_embedding_model`,
      (SELECT "This movie was bad" AS content),
      STRUCT(TRUE AS flatten_json_output)
    )
)
SELECT
  review,
  ML.DISTANCE(
    (SELECT text_embedding FROM bad_review_embedding),
    text_embedding,
    'COSINE'
  ) AS distance_to_average_review
FROM
  test_embeddings.embedded_reviews
ORDER BY distance_to_average_review
limit 10;

Query is running:   0%|          |

Downloading:   0%|          |

,review,distance_to_average_review
0,"This is a terrible movie, don't waste your mon...",0.113749
1,"This was truly horrible. Bad acting, bad writi...",0.124198
2,I watched this movie just a little while ago a...,0.147019
3,Please avoid this movie at all costs. This is ...,0.151887
4,This was one of the worst movies i have ever s...,0.159810
5,I'm not going to say too much as this movie is...,0.161036
6,This movie was a complete waste of time. The s...,0.161708
7,This was one of the worst movies I have ever s...,0.165163
8,horrible! All i can say is that is movie was h...,0.167293
9,This was a very disappointing movie. I would d...,0.167387


### Welcome to BigQuery Vector Search

Let's use the latest feature of BigQuery Vector Search to help us perform faster search. [VECTOR_SEARCH function](https://cloud.google.com/bigquery/docs/reference/standard-sql/search_functions#vector_search) uses the Approximate Nearest Neighbor search technique to help improve vector search performance, with the trade-off of reducing recall and so returning more approximate results. Brute force is used to return exact results when a vector index isn't available, and you can choose to use brute force to get exact results even when a vector index is available.

Let's start by creating a vector index. The following code creates a vector index on the text_embedding column of embedded_reviews table:

In [48]:

# create vector index
%%bigquery
CREATE VECTOR INDEX review_embedding_index ON test_embeddings.embedded_reviews(text_embedding) OPTIONS(distance_type = 'EUCLIDEAN', index_type='IVF');


Query is running:   0%|          |

""


Verify that new vector index is created correctly.

In [49]:
%%bigquery
SELECT table_name, index_name, ddl, coverage_percentage
  FROM `test_embeddings`.INFORMATION_SCHEMA.VECTOR_INDEXES
  WHERE index_status = 'ACTIVE';



Query is running:   0%|          |

Downloading:   0%|          |

,table_name,index_name,ddl,coverage_percentage
0,embedded_reviews,review_embedding_index,CREATE VECTOR INDEX `review_embedding_index` O...,0


Now we can search using Vector Search based on a specific embedded text.

To do vector search, use the VECTOR_SEARCH function. VECTOR_SEARCH is designed to be used with vector indexes to speed up vector search with the trade off on recall. It also supports vector searches using brute force without an index.

In [50]:
%%bigquery
WITH bad_review_embedding AS (
  SELECT text_embedding
  FROM
    ML.GENERATE_TEXT_EMBEDDING(
      MODEL `test_embeddings.llm_embedding_model`,
      (SELECT "This movie was bad" AS content),
      STRUCT(TRUE AS flatten_json_output)
    )
)
SELECT *
FROM VECTOR_SEARCH(
  TABLE test_embeddings.embedded_reviews,
  "text_embedding",
  (SELECT text_embedding FROM bad_review_embedding),
  top_k => 10
);


Query is running:   0%|          |

Downloading:   0%|          |

,query,base,distance
0,"{'text_embedding': [0.016791360452771187, -0.0...","{'review': 'This is a terrible movie, don't wa...",0.477459
1,"{'text_embedding': [0.016791360452771187, -0.0...",{'review': 'This was truly horrible. Bad actin...,0.499091
2,"{'text_embedding': [0.016791360452771187, -0.0...",{'review': 'I watched this movie just a little...,0.542764
3,"{'text_embedding': [0.016791360452771187, -0.0...",{'review': 'Please avoid this movie at all cos...,0.552019
4,"{'text_embedding': [0.016791360452771187, -0.0...",{'review': 'This was one of the worst movies i...,0.565644
5,"{'text_embedding': [0.016791360452771187, -0.0...",{'review': 'I'm not going to say too much as t...,0.568159
6,"{'text_embedding': [0.016791360452771187, -0.0...",{'review': 'This movie was a complete waste of...,0.569161
7,"{'text_embedding': [0.016791360452771187, -0.0...",{'review': 'This was one of the worst movies I...,0.574853
8,"{'text_embedding': [0.016791360452771187, -0.0...",{'review': 'horrible! All i can say is that is...,0.578612
9,"{'text_embedding': [0.016791360452771187, -0.0...",{'review': 'This was a very disappointing movi...,0.579157


Let's try another example with Vector Search. This time we search based on a value in the review column.

In [51]:
%%bigquery
SELECT *
FROM VECTOR_SEARCH(
  TABLE embeddings.embedded_reviews,
  "text_embedding",
  (SELECT review, text_embedding FROM embeddings.embedded_reviews where LEFT(review, 36)='Please avoid this movie at all costs' limit 1),
  top_k => 10
);

Query is running:   0%|          |

Downloading:   0%|          |

,query,base,distance
0,{'review': 'Please avoid this movie at all cos...,{'review': 'Please avoid this movie at all cos...,0.000000
1,{'review': 'Please avoid this movie at all cos...,"{'review': 'This is a terrible movie, don't wa...",0.399265
2,{'review': 'Please avoid this movie at all cos...,"{'review': 'Sorry, folks! This is without a do...",0.416670
3,{'review': 'Please avoid this movie at all cos...,{'review': 'The worst movie I've ever seen in ...,0.419622
4,{'review': 'Please avoid this movie at all cos...,{'review': 'If you are planning to rent or buy...,0.431801
5,{'review': 'Please avoid this movie at all cos...,{'review': 'This movie was by far the worst mo...,0.433852
6,{'review': 'Please avoid this movie at all cos...,{'review': 'This is the worst movie of ALL TIM...,0.439687
7,{'review': 'Please avoid this movie at all cos...,{'review': 'This was a very disappointing movi...,0.442381
8,{'review': 'Please avoid this movie at all cos...,{'review': 'I'm not going to say too much as t...,0.449759
9,{'review': 'Please avoid this movie at all cos...,{'review': 'This was one of the worst movies i...,0.455810
